In [ ]:

from scipy import signal
from scipy.integrate import simps
import numpy as np
from fooof import FOOOF
import matplotlib.pyplot as plt
import pickle

def harmonics_removal(signal, fs, harmonics, dftbandwidth=1, dftneighbourwidth=2):
    """
    Removes beta harmonics in a signal via spectrum interpolation.

    Parameters:
    - signal: 1D numpy array, the input signal
    - fs: float, sampling rate in Hz
    - harmonics: list of floats, harmonics frequencies in Hz (e.g., [50, 100, 150])
    - dftbandwidth: float, half bandwidth of harmonics frequency bands in Hz (default 1)
    - dftneighbourwidth: float, width of neighbouring frequencies in Hz (default 2)

    Returns:
    - cleaned_signal: 1D numpy array, the signal withouth the indicated beta harmonics    """
    # FFT of the signal
    N = len(signal)
    freqs = np.fft.fftfreq(N, 1/fs)
    signal_fft = np.fft.fft(signal)

    # Helper function to get indices of frequency bins
    def get_freq_indices(freq, bandwidth, fs, N):
        
        return np.where((freqs >= (freq - bandwidth)) & (freqs <= (freq + bandwidth)))[0]

    # Process each harmonic
    for f in harmonics:
        harmonics_indices = get_freq_indices(f, dftbandwidth, fs, N)
       
        harmonics_indices = np.concatenate((harmonics_indices, get_freq_indices(-f, dftbandwidth, fs, N)))
       
        for harmonics_index in harmonics_indices:
            # Find neighbouring indices
            lower_bound = f - dftneighbourwidth - dftbandwidth
            upper_bound = f + dftneighbourwidth + dftbandwidth
            neighbours = np.where((freqs >= lower_bound) & (freqs <= upper_bound) & 
                                  ((freqs < (f - dftbandwidth)) | (freqs > (f + dftbandwidth))))[0]
            
            # Compute the mean amplitude of neighbouring bins
            if len(neighbours) > 1:
                neighbour_freqs = freqs[neighbours]
          
            
                neighbour_amplitudes = np.abs(signal_fft[neighbours])
                
                interpolated_amplitude = np.mean(neighbour_amplitudes)
                original_phase = np.angle(signal_fft[harmonics_index])
                # Replace the amplitude of the harmonics frequency bin by the interpolated value
                signal_fft[harmonics_index] = interpolated_amplitude * np.exp(1j * original_phase)
    # Inverse FFT to get the cleaned signal

    cleaned_signal = np.fft.ifft(signal_fft).real
    
    return cleaned_signal

#Center of gravity method for computing central frequency
def  cog(f,pxx,f1,f2):
    prod=f*pxx
    cog=abs(simps(prod[(f1<f) & (f<f2)], f[(f1<f) & (f<f2)])/simps(pxx[(f1<f) & (f<f2)], f[(f1<f) & (f<f2)]))
    return cog

#Aperiodic function that will be fitted and removed for the computation of the spectral power

def aper(f,offset,exp):
     return 10**offset/(f**exp)


   

# GPe-TI

Computing spectral power and peak frequencies for gamma of GPe-TI and isolated GPe-TI

In [ ]:


# Load pre-saved data from a pickle file
result = pickle.load(open("isolated_gp.p", "rb"))


f1, f2 = 50, 150  # Frequency range for gamma band analysis
n_pop = 780       # Population size of neurons

# Initialize variables to store results across trials and conditions
k = 0
peak_mean, peak_std, dd, gamma_mean, gamma_std, beta_mean, beta_std, firing_mean, firing_std, tresh_gamma, tresh_beta = [], [], [], [], [], [], [], [], [], [], []
peak_mean_isol, peak_std_isol = [], []
firing_mean_isol, firing_std_isol = [], []
gamma_mean_isol, gamma_std_isol = [], []
std_isol_mean, std_mean = [], []
std_isol_std, std_std = [], []

# Iterate through different values of D_d (dopamine depletion) in the result
for D_d in result:
    if float(D_d)>1.0:
        continue
    # Initialize variables for storing trial-specific results

    t_f = 5000  # Total simulation time
    t_s = 1     # Sampling time
    nparseg = int(1000 / t_s)  # Parameter for Welch's method
    fs = 1000 / t_s            # Sampling frequency
    n_trials = 5               # Number of trials
    peak, peak_isol, gamma, beta, fr_temp, fr_temp_isol, gamma_isol = [], [], [], [], [], [], []
    std_temp, std_temp_isol = [], []

    # Loop over trials
    for i in np.arange(0, n_trials, 1):
        data, data_isol = result[D_d][i]  # Extract data

        # Compute power spectral densities (PSDs) using Welch's method
        f, pxx = signal.welch(data, fs, nperseg=nparseg, noverlap=int(nparseg / 2), nfft=max(30000, nparseg), scaling='density', window='hamming')
        f, pxx_isol = signal.welch(data_isol, fs, nperseg=nparseg, noverlap=int(nparseg / 2), nfft=max(30000, nparseg), scaling='density', window='hamming')

        # Compute beta frequency center of gravity
        f_beta = cog(f, pxx, 10, 30)

        # Compute firing rates and standard deviations (normalized by population size)
        fr_temp.append(np.mean(data) / n_pop * 1000)
        fr_temp_isol.append(np.mean(data_isol) / n_pop * 1000)
        std_temp.append(np.std(data) / n_pop * 1000)
        std_temp_isol.append(np.std(data_isol) / n_pop * 1000)

        # Apply harmonics removal high dopamine depletion (only for non-isolated case)
        if float(D_d) > 0.9:
            harmonics = np.arange(2, 6) * f_beta
            data = harmonics_removal(data, fs, harmonics, 5, 3)

        # Recompute PSDs after harmonics removal
        f, pxx = signal.welch(data, fs, nperseg=nparseg, noverlap=int(nparseg / 2), nfft=max(30000, nparseg), scaling='density', window='hamming')

        # Fit the aperiodic component using the FOOOF model
        fm = FOOOF(max_n_peaks=3, peak_width_limits=[2, 50], verbose=False)
        fm.fit(f, pxx, freq_range=[5, 500])
        pxx[f > 5] = pxx[f > 5] - aper(f[f > 5], *fm.aperiodic_params_)

        # Repeat FOOOF fitting for isolated data
        fm.fit(f, pxx_isol, freq_range=[5, 500])
        pxx_isol[f > 5] = pxx_isol[f > 5] - aper(f[f > 5], *fm.aperiodic_params_)

        # Compute center of gravity for gamma band
        f_peak = cog(f, pxx, f1, f2)
        f_peak_isol = cog(f, pxx_isol, f1, f2)

        # Gamma power for main data and isolated data
        fmin, fmax = f_peak - 20, f_peak + 20
        gamma_power = simps(pxx[(fmin < f) & (f < fmax)], f[(fmin < f) & (f < fmax)]) / (fmax - fmin)
        fmin, fmax = f_peak_isol - 20, f_peak_isol + 20
        gamma_power_isol = simps(pxx_isol[(fmin < f) & (f < fmax)], f[(fmin < f) & (f < fmax)]) / (fmax - fmin)

        # Beta power computation
        fmin_beta, fmax_beta = f_beta - 5, f_beta + 5
        beta_power = simps(pxx[(fmin_beta < f) & (f < fmax_beta)], f[(fmin_beta < f) & (f < fmax_beta)]) / (fmax_beta - fmin_beta)

        # Append trial-specific results
        peak.append(f_peak)
        peak_isol.append(f_peak_isol)
        gamma.append(gamma_power)
        beta.append(beta_power)
        gamma_isol.append(gamma_power_isol)

        # Generate Poissonian activity and compute thresholds
        if i == 4:
            gamma_power_pois = []
            beta_power_pois = []
            for n in range(100):
                pois = np.random.binomial(n_pop, np.mean(np.array(fr_temp) / 1000), len(data))
                f_poisson, Pxx_poisson = signal.welch(pois, fs, nperseg=nparseg, noverlap=int(nparseg / 2), nfft=max(30000, nparseg), scaling='density', window='hamming')
                gamma_power_pois.append(simps(Pxx_poisson[(fmin < f) & (f < fmax)], f[(fmin < f) & (f < fmax)]) / (fmax - fmin))
            treshold_gamma = np.mean(np.array(gamma_power_pois))

    
    peak_mean.append(np.mean(np.array(peak)))
    peak_std.append(np.std(np.array(peak)))
    peak_mean_isol.append(np.mean(np.array(peak_isol)))
    peak_std_isol.append(np.std(np.array(peak_isol)))

    dd.append(float(D_d))
    gamma_mean.append(np.mean(np.array(gamma)))
    gamma_std.append(np.std(np.array(gamma)))
    tresh_gamma.append(treshold_gamma)
    gamma_mean_isol.append(np.mean(np.array(gamma_isol)))
    gamma_std_isol.append(np.std(np.array(gamma_isol)))
    
    std_std.append(np.std(np.array(std_temp)))
    std_mean.append(np.mean(np.array(std_temp)))
    std_isol_std.append(np.std(np.array(std_temp_isol)))
    std_isol_mean.append(np.mean(np.array(std_temp_isol)))
  
    beta_mean.append(np.mean(np.array(beta)))
    beta_std.append(np.std(np.array(beta)))
 
    firing_mean.append(np.mean(np.array(fr_temp)))
    firing_std.append(np.std(np.array(fr_temp)))
   
    firing_mean_isol.append(np.mean(np.array(fr_temp_isol)))
    firing_std_isol.append(np.std(np.array(fr_temp_isol)))
    
    k+=1


Comparison of mean and standard deviation of the firing rates of GPe-TI and isolated GPe-TI

In [ ]:
myblue= [0, 0.5, 1]
plt.rcParams.update({'font.size': 25})
plt.rc('axes', labelsize=25)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=25)
fig=plt.figure(figsize=(18,9))
plt.subplot(1,2,1)

plt.ylim(24,50)
plt.errorbar(np.array(dd),np.array(firing_mean),np.array(firing_std),marker='.',label='GPe-TI',color=myblue, markersize=18, capsize=10)
plt.errorbar(np.array(dd),np.array(firing_mean_isol),np.array(firing_std_isol),marker='.',label='Isolated GPe-TI',color='red', markersize=18, capsize=10)


plt.ylabel(labelpad=10,ylabel='Mean firing rate [spike/s]')
plt.xlabel(labelpad=10,xlabel='$D_d$')
plt.legend(framealpha=0,)
plt.subplot(1,2,2)
plt.ylim(10,30)

plt.errorbar(np.array(dd),np.array(std_mean),np.array(std_std),marker='.',label='GPe-TI',color=myblue, markersize=18, capsize=10)
plt.errorbar(np.array(dd),np.array(std_isol_mean),np.array(std_isol_std),marker='.',label='Isolated GPe-TI',color='red', markersize=18, capsize=10)

plt.ylabel(labelpad=10,ylabel='Firing rate std [spike/s]')
plt.xlabel(labelpad=10,xlabel='$D_d$')
plt.legend(framealpha=0,)

plt.tight_layout()


Comparison of gamma peak frequencies of GPe-TI and isolated GPe-TI

In [ ]:

fig=plt.figure(figsize=(12,12))
plt.rcParams.update({'font.size': 30})
plt.rc('axes', labelsize=30)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=25)
myblue= [0, 0.5, 1]
offset=np.abs(np.array(dd)[0]-np.array(dd)[1])/5
xlim1=np.min(np.array(dd))-offset*5
xlim2=np.max(np.array(dd))+offset*5
plt.xlim([xlim1,xlim2])
threshold_value=0
plt.xlabel(labelpad=10,xlabel='$D_d$')
plt.ylabel(labelpad=10,ylabel='Peak frequency [Hz]')
plt.errorbar(np.array(dd),np.array(peak_mean),np.array(peak_std),marker='.',label='GPe-TI',color=myblue, markersize=18, capsize=10)
plt.errorbar(np.array(dd)+offset,np.array(peak_mean_isol),np.array(peak_std_isol),marker='.',label='Isolated GPe-TI',color='red', markersize=18, capsize=10)

plt.legend(framealpha=0)
plt.ylim(84,105)
fig.tight_layout()
plt.show()



Comparison of the spectral power of GPe-TI and isolated GPe-TI

In [ ]:
fig=plt.figure(figsize=(12,12))
plt.rcParams.update({'font.size': 30})
plt.rc('axes', labelsize=30)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=25)

myvlue=[0,0.5,1]
plt.errorbar(np.array(dd),np.array(gamma_mean),np.array(gamma_std),marker='.',label='Gamma, GPe-TI',color=myblue, markersize=18, capsize=10)
plt.errorbar(np.array(dd),np.array(gamma_mean_isol),np.array(gamma_std_isol),marker='.',label='Gamma, isolated GPe-TI',color='red', markersize=18, capsize=10)
plt.errorbar(np.array(dd),np.array(beta_mean),np.array(beta_std),marker='.',label='Beta, GPe-TI',color='grey', markersize=18, capsize=10)

plt.plot(np.array(dd),np.array(tresh_gamma),color='black',linestyle='--',lw=4)
plt.rcParams.update({'font.size': 25})
plt.rc('axes', labelsize=25)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=25)
plt.ylabel(labelpad=10,ylabel='Mean spectral power [a.u.]')
plt.yscale('log')
plt.xlabel(labelpad=10,xlabel='$D_d$')
plt.legend(framealpha=0,)
fig.tight_layout()
plt.show()


Comparison of the PSDs of GPe-TI and isolated GPe-TI

In [ ]:
import numpy as np
from scipy.stats import pearsonr
from scipy import signal
import matplotlib.pyplot as plt
import pickle

# Load data
result = pickle.load(open("isolated_gp.p", "rb"))

# Initialize figure
fig = plt.figure(figsize=(24, 12))
k = 1  # Subplot index

# Iterate over the result dictionary
for D_d in result:
    # Process only healhty and PD cases
    if float(D_d) == 0.75 or float(D_d) == 1:
        t_f = 5000  # Final time point
        n_trials = 5  # Number of trials

        # Update plot configurations
        plt.rcParams.update({'font.size': 30})
        plt.rc('axes', labelsize=30)
        plt.rc('xtick', labelsize=25)
        plt.rc('ytick', labelsize=25)

        # Lists to store PSD values
        Pxx_isol_final = []
        Pxx_final = []

        # Process each trial
        for i in range(n_trials):
            # Extract data
            data, data_isol = result[D_d][i]
            t_s = 1  # Sampling interval
            nperseg = 1000  # Number of samples per segment
            fs = 1000 / t_s  # Sampling frequency

            # Compute PSD for the original data
            f, pxx = signal.welch(data, fs, nperseg=nperseg, noverlap=nperseg // 2, 
                                   nfft=max(30000, nperseg), scaling='density', window='hamming')
        

            # Compute beta frequency and remove harmonics
            f_beta = cog(f, pxx, 10, 30)
            if float(D_d) == 1:
                    harmonics = np.arange(2, 6) * f_beta
                    data = harmonics_removal(data, fs, harmonics, 5, 3)

            # Compute PSD for isolated and actual data
            f, Pxx_den_isol = signal.welch(data_isol, fs, nperseg=nperseg, noverlap=nperseg // 2, 
                                           nfft=max(30000, nperseg), scaling='density', window='hamming')
            f, Pxx = signal.welch(data, fs, nperseg=nperseg, noverlap=nperseg // 2, 
                                          nfft=max(30000, nperseg), scaling='density', window='hamming')

            # Store PSD values for specified frequency range
            Pxx_isol_final.append(Pxx_den_isol[(f > 5) & (f < 250)])
            Pxx_final.append(Pxx[(f > 5) & (f < 250)])

        # Filter frequency range for plotting
        f = f[(f > 5) & (f < 250)]

        # Plot PSDs
        plt.subplot(1, 2, k)
        if k == 1:
            plt.ylabel('PSD [a.u.]', labelpad=12)
        plt.yscale('log')
        plt.ylim([10**-1.7, 10**2.8])

        # Plot isolated data PSD
        mean_pxx_isol = np.mean(Pxx_isol_final, axis=0)
        std_pxx_isol = np.std(Pxx_isol_final, axis=0)
        plt.plot(f, mean_pxx_isol, label='Isolated GPe-TI', color='red')
        plt.fill_between(f, mean_pxx_isol - std_pxx_isol, mean_pxx_isol + std_pxx_isol, alpha=0.5, color='red')

        # Plot total data PSD
        mean_pxx = np.mean(Pxx_final, axis=0)
        std_pxx = np.std(Pxx_final, axis=0)
        plt.plot(f, mean_pxx, label='GPe-TI', color='blue')
        plt.fill_between(f, mean_pxx - std_pxx, mean_pxx + std_pxx, alpha=0.5, color='blue')

        # Add legend
        legend = plt.legend(loc='upper right', framealpha=0)
        for line in legend.get_lines():
            line.set_linewidth(4)

        # Calculate and display correlation
        corr, _ = pearsonr(mean_pxx[(f > 50) & (f < 150)], mean_pxx_isol[(f > 50) & (f < 150)])
        plt.title(f'{"Healthy" if float(D_d) == 0.75 else "PD"}, $R^2$={corr**2:.2f}')
        plt.xlabel('Frequency [Hz]', labelpad=12)

        k += 1  # Increment subplot index

# Finalize and show figure
fig.tight_layout()
plt.show()


# D2

Computing spectral power and peak frequencies for gamma of D2 and isolated D2

In [30]:


# Load pre-saved data from a pickle file
result = pickle.load(open("isolated_d2.p", "rb"))


f1, f2 = 40, 120  # Frequency range for gamma band analysis
n_pop = 6000       # Population size of neurons

# Initialize variables to store results across trials and conditions
k = 0
peak_mean, peak_std, dd, gamma_mean, gamma_std, beta_mean, beta_std, firing_mean, firing_std, tresh_gamma, tresh_beta = [], [], [], [], [], [], [], [], [], [], []
peak_mean_isol, peak_std_isol = [], []
firing_mean_isol, firing_std_isol = [], []
gamma_mean_isol, gamma_std_isol = [], []
std_isol_mean, std_mean = [], []
std_isol_std, std_std = [], []

# Iterate through different values of D_d (dopamine depletion) in the result
for D_d in result:
    if float(D_d)>1.0:
        continue
    # Initialize variables for storing trial-specific results

    t_f = 5000  # Total simulation time
    t_s = 1     # Sampling time
    nparseg = int(1000 / t_s)  # Parameter for Welch's method
    fs = 1000 / t_s            # Sampling frequency
    n_trials = 5               # Number of trials
    peak, peak_isol, gamma, beta, fr_temp, fr_temp_isol, gamma_isol = [], [], [], [], [], [], []
    std_temp, std_temp_isol = [], []

    # Loop over trials
    for i in np.arange(0, n_trials, 1):
        data, data_isol = result[D_d][i]  # Extract data

        # Compute power spectral densities (PSDs) using Welch's method
        f, pxx = signal.welch(data, fs, nperseg=nparseg, noverlap=int(nparseg / 2), nfft=max(30000, nparseg), scaling='density', window='hamming')
        f, pxx_isol = signal.welch(data_isol, fs, nperseg=nparseg, noverlap=int(nparseg / 2), nfft=max(30000, nparseg), scaling='density', window='hamming')

        # Compute beta frequency center of gravity
        f_beta = cog(f, pxx, 10, 30)

        # Compute firing rates and standard deviations (normalized by population size)
        fr_temp.append(np.mean(data) / n_pop * 1000)
        fr_temp_isol.append(np.mean(data_isol) / n_pop * 1000)
        std_temp.append(np.std(data) / n_pop * 1000)
        std_temp_isol.append(np.std(data_isol) / n_pop * 1000)

        # Apply harmonics removal high dopamine depletion (only for non-isolated case)
        if float(D_d) > 0.9:
            harmonics = np.arange(2, 4) * f_beta
            data = harmonics_removal(data, fs, harmonics, 5, 3)

        # Recompute PSDs after harmonics removal
        f, pxx = signal.welch(data, fs, nperseg=nparseg, noverlap=int(nparseg / 2), nfft=max(30000, nparseg), scaling='density', window='hamming')

        # Fit the aperiodic component using the FOOOF model
        fm = FOOOF(max_n_peaks=3, peak_width_limits=[2, 50], verbose=False)
        fm.fit(f, pxx, freq_range=[5, 500])
        pxx[f > 5] = pxx[f > 5] - aper(f[f > 5], *fm.aperiodic_params_)

        # Repeat FOOOF fitting for isolated data
        fm.fit(f, pxx_isol, freq_range=[5, 500])
        pxx_isol[f > 5] = pxx_isol[f > 5] - aper(f[f > 5], *fm.aperiodic_params_)

        # Compute center of gravity for gamma band
        f_peak = cog(f, pxx, f1, f2)
        f_peak_isol = cog(f, pxx_isol, f1, f2)

        # Gamma power for main data and isolated data
        fmin, fmax = f_peak - 20, f_peak + 20
      
        gamma_power = simps(pxx[(fmin < f) & (f < fmax)], f[(fmin < f) & (f < fmax)]) / (fmax - fmin)
        fmin, fmax = f_peak_isol - 20, f_peak_isol + 20
        gamma_power_isol = simps(pxx_isol[(fmin < f) & (f < fmax)], f[(fmin < f) & (f < fmax)]) / (fmax - fmin)

        # Beta power computation
        fmin_beta, fmax_beta = f_beta - 5, f_beta + 5
        beta_power = simps(pxx[(fmin_beta < f) & (f < fmax_beta)], f[(fmin_beta < f) & (f < fmax_beta)]) / (fmax_beta - fmin_beta)

        # Append trial-specific results
        peak.append(f_peak)
        peak_isol.append(f_peak_isol)
        gamma.append(gamma_power)
        beta.append(beta_power)
        gamma_isol.append(gamma_power_isol)

        # Generate Poissonian activity and compute thresholds
        if i == 4:
            gamma_power_pois = []
            beta_power_pois = []
            for n in range(100):
                pois = np.random.binomial(n_pop, np.mean(np.array(fr_temp) / 1000), len(data))
                f_poisson, Pxx_poisson = signal.welch(pois, fs, nperseg=nparseg, noverlap=int(nparseg / 2), nfft=max(30000, nparseg), scaling='density', window='hamming')
                gamma_power_pois.append(simps(Pxx_poisson[(fmin < f) & (f < fmax)], f[(fmin < f) & (f < fmax)]) / (fmax - fmin))
            treshold_gamma = np.mean(np.array(gamma_power_pois))

    
    peak_mean.append(np.mean(np.array(peak)))
    peak_std.append(np.std(np.array(peak)))
    peak_mean_isol.append(np.mean(np.array(peak_isol)))
    peak_std_isol.append(np.std(np.array(peak_isol)))

    dd.append(float(D_d))
    gamma_mean.append(np.mean(np.array(gamma)))
    gamma_std.append(np.std(np.array(gamma)))
    tresh_gamma.append(treshold_gamma)
    gamma_mean_isol.append(np.mean(np.array(gamma_isol)))
    gamma_std_isol.append(np.std(np.array(gamma_isol)))
    
    std_std.append(np.std(np.array(std_temp)))
    std_mean.append(np.mean(np.array(std_temp)))
    std_isol_std.append(np.std(np.array(std_temp_isol)))
    std_isol_mean.append(np.mean(np.array(std_temp_isol)))
  
    beta_mean.append(np.mean(np.array(beta)))
    beta_std.append(np.std(np.array(beta)))
 
    firing_mean.append(np.mean(np.array(fr_temp)))
    firing_std.append(np.std(np.array(fr_temp)))
   
    firing_mean_isol.append(np.mean(np.array(fr_temp_isol)))
    firing_std_isol.append(np.std(np.array(fr_temp_isol)))
    
    k+=1



Comparison of averages and standard deviations of the firing rates of D2 and isolated D2 

In [ ]:
plt.rcParams.update({'font.size': 25})
plt.rc('axes', labelsize=25)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=25)
fig=plt.figure(figsize=(18,9))
plt.subplot(1,2,1)


plt.errorbar(np.array(dd),np.array(firing_mean),np.array(firing_std),marker='.',label='D2',color=myblue, markersize=18, capsize=10)
plt.errorbar(np.array(dd),np.array(firing_mean_isol),np.array(firing_std_isol),marker='.',label='Isolated D2',color='red', markersize=18, capsize=10)
plt.ylim(0,5)

plt.ylabel(labelpad=10,ylabel='Mean firing rate [spike/s]')
plt.xlabel(labelpad=10,xlabel='$D_d$')
plt.legend(framealpha=0,)
plt.subplot(1,2,2)


plt.errorbar(np.array(dd),np.array(std_mean),np.array(std_std),marker='.',label='D2',color=myblue, markersize=18, capsize=10)
plt.errorbar(np.array(dd),np.array(std_isol_mean),np.array(std_isol_std),marker='.',label='Isolated D2',color='red', markersize=18, capsize=10)

plt.ylabel(labelpad=10,ylabel='Firing rate std [spike/s]')
plt.xlabel(labelpad=10,xlabel='$D_d$')
plt.legend(framealpha=0,)
plt.ylim(0,2)
plt.tight_layout()


Comparison of D2 and isolated D2 spectral power, across dopamine depletion levels

In [ ]:
fig=plt.figure(figsize=(12,12))
plt.rcParams.update({'font.size': 30})
plt.rc('axes', labelsize=30)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=25)
myblue=[0,0.5,1]

plt.errorbar(np.array(dd),np.array(gamma_mean),np.array(gamma_std),marker='.',label='Gamma, D2',color=myblue, markersize=18, capsize=10)
plt.errorbar(np.array(dd),np.array(gamma_mean_isol),np.array(gamma_std_isol),marker='.',label='Gamma, isolated D2',color='red', markersize=18, capsize=10)
plt.errorbar(np.array(dd),np.array(beta_mean),np.array(beta_std),marker='.',label='Beta, D2',color='grey', markersize=18, capsize=10)

plt.plot(np.array(dd),np.array(tresh_gamma),color='black',linestyle='--',lw=4)
plt.rcParams.update({'font.size': 25})
plt.rc('axes', labelsize=25)
plt.rc('xtick', labelsize=25)
plt.rc('ytick', labelsize=25)
plt.ylabel(labelpad=10,ylabel='Mean spectral power [a.u.]')
plt.yscale('log')
plt.xlabel(labelpad=10,xlabel='$D_d$')
plt.legend(framealpha=0,)
fig.tight_layout()


Comparison of D2 and isolated D2 power spectral densities, in healthy and pathological cases

In [ ]:
import numpy as npf
from scipy.stats import pearsonr
from scipy import signal
import matplotlib.pyplot as plt
import pickle

# Load data
result = pickle.load(open("isolated_d2.p", "rb"))

# Initialize figure
fig = plt.figure(figsize=(24, 12))
k = 1  # Subplot index

# Iterate over the result dictionary
for D_d in result:
    # Process only specific D_d values
    if float(D_d) == 0.75 or float(D_d) == 1:
        t_f = 5000  # Final time point
        n_trials = 5  # Number of trials

        # Update plot configurations
        plt.rcParams.update({'font.size': 30})
        plt.rc('axes', labelsize=30)
        plt.rc('xtick', labelsize=25)
        plt.rc('ytick', labelsize=25)

        # Lists to store PSD values
        Pxx_isol_final = []
        Pxx_final = []

        # Process each trial
        for i in range(n_trials):
            # Extract data
            data, data_isol = result[D_d][i]
            t_s = 1  # Sampling interval
            nperseg = 1000  # Number of samples per segment
            fs = 1000 / t_s  # Sampling frequency

            # Compute PSD for the original data
            f, pxx = signal.welch(data, fs, nperseg=nperseg, noverlap=nperseg // 2, 
                                   nfft=max(30000, nperseg), scaling='density', window='hamming')
        

            # Compute beta frequency and remove harmonics
            f_beta = cog(f, pxx, 10, 30)
            if float(D_d) == 1:
                    harmonics = np.arange(2, 5) * f_beta
                    data = harmonics_removal(data, fs, harmonics, 5, 3)

            # Compute PSD for isolated and actual data
            f, Pxx_den_isol = signal.welch(data_isol, fs, nperseg=nperseg, noverlap=nperseg // 2, 
                                           nfft=max(30000, nperseg), scaling='density', window='hamming')
            f, Pxx = signal.welch(data, fs, nperseg=nperseg, noverlap=nperseg // 2, 
                                          nfft=max(30000, nperseg), scaling='density', window='hamming')

            # Store PSD values for specified frequency range
            Pxx_isol_final.append(Pxx_den_isol[(f > 5) & (f < 250)])
            Pxx_final.append(Pxx[(f > 5) & (f < 250)])

        # Filter frequency range for plotting
        f = f[(f > 5) & (f < 250)]

        # Plot PSDs
        plt.subplot(1, 2, k)
        if k == 1:
            plt.ylabel('PSD [a.u.]', labelpad=12)
        plt.yscale('log')
        plt.ylim([10**-1.7, 10**2.8])

        # Plot isolated data PSD
        mean_pxx_isol = np.mean(Pxx_isol_final, axis=0)
        std_pxx_isol = np.std(Pxx_isol_final, axis=0)
        plt.plot(f, mean_pxx_isol, label='Isolated D2', color='red')
        plt.fill_between(f, mean_pxx_isol - std_pxx_isol, mean_pxx_isol + std_pxx_isol, alpha=0.5, color='red')

        # Plot total data PSD
        mean_pxx = np.mean(Pxx_final, axis=0)
        std_pxx = np.std(Pxx_final, axis=0)
        plt.plot(f, mean_pxx, label='D2', color='blue')
        plt.fill_between(f, mean_pxx - std_pxx, mean_pxx + std_pxx, alpha=0.5, color='blue')

        # Add legend
        legend = plt.legend(loc='upper right', framealpha=0)
        for line in legend.get_lines():
            line.set_linewidth(4)
        plt.ylim([10**(-3),10**(2)])
        # Calculate and display correlation
        corr, _ = pearsonr(mean_pxx[(f > 50) & (f < 150)], mean_pxx_isol[(f > 50) & (f < 150)])
        plt.title(f'{"Healthy" if float(D_d) == 0.75 else "PD"}, $R^2$={corr**2:.2f}')
        plt.xlabel('Frequency [Hz]', labelpad=12)

        k += 1  # Increment subplot index

# Finalize and show figure
fig.tight_layout()
plt.show()
